In [14]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("Key_Value_RDD_OP")
sc = SparkContext(conf=conf)
sc

23/11/15 14:13:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


<SparkContext master=local appName=Key_Value_RDD_OP>

# groupByKey
* `k_v_RDD.groupByKey(numPartitions=None, partitionFunc=<function portable_hash>)`
* 주어지는 key를 기준으로 Group을 만들어 줍니다.
* Transformations 함수 입니다.

In [15]:
rdd = sc.parallelize([
    ("짜장면", 15),
    ("짬뽕", 10),
    ("짜장면", 5)
])

rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [16]:
rdd.groupByKey()

PythonRDD[5] at RDD at PythonRDD.scala:53

In [17]:
# 각 그룹 별로 몇 개의 원소가 있는지 확인
# "짜장면"은 그래로 두고 숫자의 value만 확인한다 => mapValues를 사용해서

rdd.groupByKey().mapValues(len).collect()

[('짜장면', 2), ('짬뽕', 1)]

In [18]:
rdd.groupByKey().mapValues(list).collect()

[('짜장면', [15, 5]), ('짬뽕', [10])]

# reduceByKey

- keyvalueRdd, reduceByKey(<task>, numPartitions = None, partitionFunc = <function portable_hash>)
- 주어지는 key를 기준으로 Group을 만들고 합쳐 (task대로) 줍니다.
- Transformations 함수입니다.    

key로 value값을 모아서 연산하는거 =>  reduceByKey

In [19]:
rdd.reduceByKey(lambda x, y : x + y).collect()

[('짜장면', 20), ('짬뽕', 10)]

# mapValues
- `KeyValueRDD.mapValues(<task>)`
- 함수를 `Value`에만 적용합니다.
    - 파티션과 키는 그 위치에 그대로 있습니다.
- `Transformations`

In [20]:
rdd = sc.parallelize([
    ("하의", ["청바지", "반바지", "치마"]),
    ("상의", ["니트", "반팔", "긴팔", "나시"])
])

In [21]:
rdd.mapValues(len).collect()

[('하의', 3), ('상의', 4)]

# countByKey
- `KeyValueRDD.countByKey(<task>)`
- 각 키가 가진 요소들의 개수를 센다.
- `Action`

In [22]:
rdd.countByKey()

defaultdict(int, {'하의': 1, '상의': 1})

# Join
- `Inner Join` : 서로간에 존재하는 키만 합쳐준다.
- `Outer Join` : 기준이 되는 한 쪽에는 데이터가 있고, 다른 쪽에는 데이터가 없는 경우
    - 설정한 기준에 따라서 기준에 맞는 데이터가 항상 남아있는다.
    - `leftOuterJoin` : 왼쪽에 있는 rdd가 기준이 된다.( 함수를 호출 하는 쪽 )
    - `rightOuterJoin` : 오른쪽에 있는 rdd가 기준이 된다. ( 함수에 매개변수로 들어가는 쪽 )

In [23]:
rdd1 = sc.parallelize([
    ("foo", 1),
    ("goo", 2),
    ("hoo", 3)
])

rdd2 = sc.parallelize([
    ("foo", 1),
    ("goo", 2),
    ("goo", 10),
    ("moo", 6)
])

In [24]:
# Inner Join
rdd1.join(rdd2).collect()

[('foo', (1, 1)), ('goo', (2, 2)), ('goo', (2, 10))]

In [25]:
# Left Outer Join : join을 호출하는 쪽이 Left, 매개변수로 들어가는 쪽이 Right
rdd1.leftOuterJoin(rdd2).collect()

[('foo', (1, 1)), ('goo', (2, 2)), ('goo', (2, 10)), ('hoo', (3, None))]

In [26]:
sc.stop()